<a href="https://colab.research.google.com/github/AlishShresth/DS-Case-Study/blob/main/RealEstatePricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing modules
import numpy as np
import pandas as pd


In [5]:
# importing data file
url = 'https://raw.githubusercontent.com/AlishShresth/DS-Case-Study/main/Bengaluru_House_Data.xls'
data = pd.read_csv(url)

In [7]:
# checking data
data.head()


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [11]:
# dropping columns
data.drop(columns=['area_type','availability','society','balcony'],inplace=True)


In [16]:
data['location'].value_counts()

Whitefield                        540
Sarjapur  Road                    399
Electronic City                   302
Kanakpura Road                    273
Thanisandra                       234
                                 ... 
Bapuji Layout                       1
1st Stage Radha Krishna Layout      1
BEML Layout 5th stage               1
singapura paradise                  1
Abshot Layout                       1
Name: location, Length: 1305, dtype: int64

In [17]:
# filling null location cells with 'Thanisandra'
data['location'] = data['location'].fillna('Thanisandra')

In [21]:
# filling null size cells with '5 BHK'
data['size'] = data['size'].fillna('5 BHK')

In [23]:
# to check how many non-null values are there
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    13320 non-null  object 
 1   size        13320 non-null  object 
 2   total_sqft  13320 non-null  object 
 3   bath        13247 non-null  float64
 4   price       13320 non-null  float64
dtypes: float64(2), object(3)
memory usage: 520.4+ KB


In [25]:
# filling null bath cells with '4.0'
data['bath'] = data['bath'].fillna(4.0)

In [27]:
# creating new coloumn bhk and storing size value with bhk as integer
data['bhk'] = data['size'].str.split().str.get(0).astype(int) 

In [28]:
data['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [29]:
# to convert range value in total_sqft column
def convertRange(x):
  temp = x.split('-')
  if len(temp)==2:
    return(float(temp[0])+float(temp[1]))/2
  try:
    return float(x)
  except:
    return None

In [30]:
data['total_sqft']=data['total_sqft'].apply(convertRange)

In [32]:
# creating new column price per sqft
data['price_per_sqft'] = data['price'] * 100000 / data['total_sqft']

In [35]:
data['location'].value_counts()

Whitefield                        540
Sarjapur  Road                    399
Electronic City                   302
Kanakpura Road                    273
Thanisandra                       235
                                 ... 
Bapuji Layout                       1
1st Stage Radha Krishna Layout      1
BEML Layout 5th stage               1
singapura paradise                  1
Abshot Layout                       1
Name: location, Length: 1305, dtype: int64

In [36]:
# locations having values less than or equal to 10
data['location'] = data['location'].apply(lambda x: x.strip())
location_count = data['location'].value_counts()

In [37]:
location_count_less_10 = location_count[location_count<=10]
location_count_less_10

Dairy Circle                      10
Nagappa Reddy Layout              10
Basapura                          10
1st Block Koramangala             10
Sector 1 HSR Layout               10
                                  ..
Bapuji Layout                      1
1st Stage Radha Krishna Layout     1
BEML Layout 5th stage              1
singapura paradise                 1
Abshot Layout                      1
Name: location, Length: 1053, dtype: int64

In [38]:
# replacing locations having values less than or equal to 10 to 'other'
data['location'] = data['location'].apply(lambda x: 'other' if x in location_count_less_10 else x)

In [40]:
data['location'].value_counts()

other                 2885
Whitefield             541
Sarjapur  Road         399
Electronic City        304
Kanakpura Road         273
                      ... 
Nehru Nagar             11
Banjara Layout          11
LB Shastri Nagar        11
Pattandur Agrahara      11
Narayanapura            11
Name: location, Length: 242, dtype: int64

In [41]:
# detecting and removing outliers
data = data[((data['total_sqft']/data['bhk'])>=300)]
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,12528.000000,12528.000000,12528.000000,12528.000000,12528.000000
mean,1594.595365,2.570961,111.398192,2.654294,6304.845193
std,1261.369449,1.082563,152.084324,0.979626,4162.004738
min,300.000000,1.000000,9.000000,1.000000,267.829813
25%,1116.000000,2.000000,49.000000,2.000000,4210.526316
50%,1300.000000,2.000000,70.000000,3.000000,5294.117647
75%,1700.000000,3.000000,115.000000,3.000000,6916.666667
max,52272.000000,16.000000,3600.000000,16.000000,176470.588235


In [42]:
def remove_outliers_sqft(df):
  df_output = pd.DataFrame()
  for key, subdf in df.groupby('location'):
    m = np.mean(subdf.price_per_sqft)
    st = np.std(subdf.price_per_sqft)
    gen_df = subdf[(subdf.price_per_sqft>(m-st))&(subdf.price_per_sqft<=(m+st))]
    df_output = pd.concat([df_output,gen_df],ignore_index = True)
  return df_output
data = remove_outliers_sqft(data)
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,10299.000000,10299.000000,10299.000000,10299.000000,10299.000000
mean,1508.470017,2.482571,91.293807,2.576270,5659.402034
std,880.775939,0.985308,86.349511,0.899215,2265.863688
min,300.000000,1.000000,10.000000,1.000000,1250.000000
25%,1110.000000,2.000000,49.000000,2.000000,4246.928130
50%,1286.000000,2.000000,67.000000,2.000000,5175.983437
75%,1650.000000,3.000000,100.000000,3.000000,6428.571429
max,30400.000000,16.000000,2200.000000,16.000000,24509.803922


In [43]:
def bhk_outlier_remover(df):
  exclude_indices = np.array([])
  for location, location_df in df.groupby('location'):
    bhk_stats = {}
    for bhk, bhk_df in location_df.groupby('bhk'):
      bhk_stats[bhk] = {
          'mean': np.mean(bhk_df.price_per_sqft),
          'std': np.std(bhk_df.price_per_sqft),
          'count': bhk_df.shape[0]
      }
    for bhk, bhk_df in location_df.groupby('bhk'):
      stats = bhk_stats.get(bhk-1)
      if stats and stats['count']>5:
        exclude_indices = np.append(exclude_indices,bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
  return df.drop(exclude_indices, axis = 'index')

In [44]:
data=bhk_outlier_remover(data)

In [45]:
data.shape

(7361, 7)

In [46]:
data


,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,1st Block Jayanagar,4 BHK,2850.0,4.0,428.0,4,15017.543860
1,1st Block Jayanagar,3 BHK,1630.0,3.0,194.0,3,11901.840491
2,1st Block Jayanagar,3 BHK,1875.0,2.0,235.0,3,12533.333333
3,1st Block Jayanagar,3 BHK,1200.0,2.0,130.0,3,10833.333333
4,1st Block Jayanagar,2 BHK,1235.0,2.0,148.0,2,11983.805668
...,...,...,...,...,...,...,...
10290,other,2 BHK,1200.0,2.0,70.0,2,5833.333333
10291,other,1 BHK,1800.0,1.0,200.0,1,11111.111111
10294,other,2 BHK,1353.0,2.0,110.0,2,8130.081301
10295,other,1 Bedroom,812.0,1.0,26.0,1,3201.970443


In [47]:
data.drop(columns=['size','price_per_sqft'],inplace = True)

In [48]:
data.head()

,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,1st Block Jayanagar,1235.0,2.0,148.0,2


In [49]:
data.to_csv("cleaned_data.csv")